**Proposed use:**
Testing python package installation against AI Platform Training

In [ ]:
#auth notebook
export GOOGLE_APPLICATION_CREDENTIALS="PATH"

In [ ]:
#define args & filenames
#ASSIGN/REPLACE VALUES HERE
JOB_NAME_PREFIX=""
PIP_PKG_NAME_LIST=["pkg1", "pkg2", "pkg3"]
JOB_DIR="gs://BUCKET_NAME/PATH"
RUNTIME_VERSION=""
PYTHON_VERSION=""
REGION=""

In [ ]:
%%bash
mkdir -p trainer
echo -n "" trainer/__init__.py

In [ ]:
import os, time

t = time.localtime()
t_str = time.strftime("%H%M%S", t)
JOB_NAME=JOB_NAME_PREFIX + t_str

if os.path.exists('requirements.txt'):
  os.remove('requirements.txt')
if os.path.exists('trainer/' + 'train.py'):
  os.remove('trainer/' + 'train.py')
with open('requirements.txt', 'a') as reqfile, open('trainer/' + 'train.py', 'a') as trainfile:
    if len(PIP_PKG_NAME_LIST) == 0:
        reqfile.write("")
    elif len(PIP_PKG_NAME_LIST) == 1:
        reqfile.write(PIP_PKG_NAME_LIST[0])
        trainfile.write("import" + " " + PIP_PKG_NAME_LIST[0])
        reqfile.write("\n")
        trainfile.write("\n")
    else:
        trainfile.write("import" + " ")
        for index in range(0,len(PIP_PKG_NAME_LIST)):
            reqfile.write(PIP_PKG_NAME_LIST[index])
            trainfile.write(PIP_PKG_NAME_LIST[index] + " ")
            if index < len(PIP_PKG_NAME_LIST) - 1:
                reqfile.write("\n")
        trainfile.write("\n")
reqfile.close()
trainfile.close()

In [ ]:
%%writefile -a trainer/train.py


if __name__ == '__main__':
   
    print("hello world!")

In [ ]:
pip install -r requirements.txt

In [ ]:
!gcloud ai-platform local train \
  --package-path trainer \
  --module-name trainer.train \
  --job-dir local-training-output

In [ ]:
!gcloud ai-platform jobs submit training $JOB_NAME \
  --job-dir $JOB_DIR \
  --package-path trainer \
  --module-name trainer.train \
  --runtime-version $RUNTIME_VERSION \
  --python-version $PYTHON_VERSION \
  --region $REGION